# My Pure Pix2Pix Implementation

****



## General Presets

In [2]:
""" Import Packages """

# Numerical Operations
import random
import numpy as np
# Reading/Writing/Cleaning Data
import pandas as pd
from PIL import Image
import os
import gc
# For Progress Bar
from tqdm.auto import tqdm
# For Display
import matplotlib.pyplot as plt
import wandb as wb
# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset, random_split
from torchvision.datasets import DatasetFolder, VisionDataset
# my_utilities
import my_utilities


In [3]:
""" For Display """

plt.style.use(r'C:\Users\chael\.matplotlib\violet.mplstyle')
wb.init(project="pure_pix2pix", name="run0")


wandb: Currently logged in as: cullencatniss. Use `wandb login --relogin` to force relogin


In [4]:
""" For Auto-reload Modules"""

%reload_ext autoreload
%autoreload 2

In [5]:
""" Assure Reproducibility """

seed=29
my_utilities.assure_reproduce(seed)

## Configurations

In [6]:
""" Path & Device"""

dataset_name = 'facades'
task_name = 'facades_pix2pix'
data_dir, checkpoints_dir , results_dir = 'datasets/'+dataset_name, 'checkpoints/'+task_name, 'results/'+task_name
device=my_utilities.register_device()

In [7]:
""" Loading & saving Rules """

# 'data_load_threads' =
# 'epoch_load' =
model_load = 'by iters.' # | by_epochs
results_save = 5000 # frequency of saving latest result (by iters.)
checkpoints_save = 5 # frequency of saving latest checkpoints (by epochs)
model_save = 'by iters.' # | by_epochs

In [8]:
""" Preprocessing Rules """

preproc = {
    'trans': 'scale & crop', # | crop | scale width | scale width & crop | none
    'scale_size': 286, # !!! in TEST TIME set to crop_size !!!
    'crop_size': 256,
    'flip': False, # whether to flip images in augmentation
}

In [9]:
""" Model """

model_name = 'pix2pix'
translate_dirct = 'facades->photo' # | 'photo->facades'

hyper_params = {
    # ======= network architectures =======
    'G_arch': 'U-net256', # | U-net128
    'D_arch': 'PatchGAN', # (for experiments in the paper) | PixelGAN | StdCNN
    # 'G_layers':
    'D_layers': 3,
    'in_chan': 3,
    'out_chan': 3,
    'G_fil_lst': 64, # num. of filters in G's last conv. layer
    'D_fil_fst': 64, # num. of filters in D's first conv. layer
    'batch_size': 32,
    'norm_type': 'instance', # (for experiments in the paper) | batch | none
    'G_dropout': False, # no dropout for G in the paper
    'D_dropout': True,
    # ============== training ==============
    # 'init_type':
    'init_scaler': 0.02, # scaling factor for weight initialization
    'shuffle_batch': True,
    'continue_train': True, # whether to load latest model
    'loss_mode': 'vanilla GAN', # | lsgan | wgangp
    'L1_lambda': 100, # L1's scaler (add L1 to GAN-loss according to the paper)
    'beta1': 0.5, # momentum for Adam (0.9?)
    # 'beta2': (?)
    'lr': 0.0002, # initial learning rate for Adam (0.001?)
    'lr_dec_mode': 'linear', # | step | plateau | cosine
    'lr_dec_iters': 50, # multiply by a gamma every lr_decay_iters iterations
    'init_lr_epochs': 100,
    'decay_lr_epochs': 100, # linear decay to 0
}

In [11]:
wb.config.update({'translate_dirct': translate_dirct, **preproc, **hyper_params})
my_utilities.print_config(device, model_load, model_save, model_name, translate_dirct, {**preproc, **hyper_params})

============CONFIGURATIONS============
Device = cuda
Model is loaded by iters.
Model is saved by iters.
Model = pix2pix
Translation Dirct. = facades->photo
--------------------------------------
Transforms Opt. = scale & crop
Img. Scale Size = 286
Img. Crop Size = 256
Flipped = False
--------------------------------------
G's Arch. = U-net256
D's Arch. = PatchGAN
num. of D's layers = 3
num. of Input's Channels = 3
num. of Output's Channels = 3
--------------------------------------
Batch Size = 32
Normalization = instance
G's Dropout = False
D's Dropout = True
--------------------------------------
Loss Mode = vanilla GAN
L1's Lambda = 100
Beta1 = 0.5
Initial Learning-rate = 0.0002
Decay Mode = linear
